In [1]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time

In [2]:
def discretize(D, b):    
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD
def count(D, a):
    counter = 0
    n = len(D)
    low = 0
    up=n-1
    mid = int((low+up)/2)
    while True:
        if D[mid]>a:
            up=mid
            mid = int((low+up)/2)
        if D[mid]<a:
            low=mid
            mid = int((low+up)/2)
        if D[mid]==a:
            i=0
            while D[mid+i]==a:
                i+=1
            return mid+i

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else: 
                low = mid
            mid = (high+low)/2
            
        return high
    if y==1/2:
        return 0
    
def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b   

In [3]:
def finiteDomainMedian(eps, beta, a,b, D):
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#score changing point
    l[0] = a
    for i in range(n+1):
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]
            
    l[n+1] = b
    u[n+1] = -n-1-u[0]
    #merge to delete duplicates
    u_merge = []
    l_merge = []
    for i in range(int(n/2)+1):
        if l[i+1]==l[i]:
            continue
        else:
            l_merge.append(l[i])
            u_merge.append(u[i])

    temp=int(n/2)+1
    idx =0
    for i in range(int(n/2)+1,n+1):
        if l[i+1]==l[i]:
            continue
        else:
            l_merge.append(l[i])
            u_merge.append(u[temp])
            if temp==int(n/2)+1:
                idx = len(l_merge)
            temp = i+1
            
    l_merge.append(b)
    u_merge.append( -n-1-u[0])
    weight = []
    for i in range(idx-1):
        weight.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u_merge[i]/2))
    weight.append(1)
    for i in range(idx,len(l_merge)):
        weight.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u_merge[i]/2))

    totalWeight = sum(weight)
    weight = weight/totalWeight
    #print(l_merge[idx-10:idx+10])
    #print(weight[idx-10:idx+10])
    i = np.random.choice(list(range(len(l_merge))), p=weight)
    if i==idx:
        return l_merge[i]
    if i<idx:
        return np.random.randint(l_merge[i], l_merge[i+1])
    if i>idx:
        return np.random.randint(l_merge[i-1], l_merge[i])

In [4]:
def EMMedian(eps, beta,  D):
    n = len(D)
    rad = 1000000
    est = finiteDomainMedian(eps, beta, -rad,rad, D)
    
    #define utility function
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#score changing point
    l[0] = -rad
    for i in range(n+1):
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]
            
    l[n+1] = rad
    u[n+1] = -n-1-u[0]
    #define u1
    factor = int(9/eps*np.log(4*rad/beta))+1
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2):
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)
          
    u1_merge = []
    l_merge = []
    for i in range(int(n/2)+1-factor):
        if l[i+1]==l[i]:
            continue
        else:
            l_merge.append(l[i])
            u1_merge.append(u1[i])

    temp=int(n/2)+1-factor
    idx =0
    for i in range(int(n/2)+1-factor,n+1):
        if l[i+1]==l[i]:
            continue
        else:
            l_merge.append(l[i])
            u1_merge.append(u1[temp])
            if temp==int(n/2)+1-factor:
                idx = len(l_merge)
            temp = i+1
            
    l_merge.append(rad)
    u1_merge.append( -n-1-u1[0])

    weight1 = []
    for i in range(idx-1):
        weight1.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u1_merge[i]/2))
    weight1.append(1)
    for i in range(idx,len(l_merge)):
        weight1.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u1_merge[i]/2))

    totalWeight1 = sum(weight1)
    weight1 = weight1/totalWeight1
    i = np.random.choice(list(range(len(l_merge))), p=weight1)
    if i==idx:
        x1= l_merge[i]
    if i<idx:
        x1= np.random.randint(l_merge[i], l_merge[i+1])
    if i>idx:
        x1= np.random.randint(l_merge[i-1], l_merge[i])
        
    u2_merge  = []  
    for i in range(int(n/2)+1+factor):
        if l[i+1]==l[i]:
            continue
        else:
            u2_merge.append(u2[i])

    temp=int(n/2)+1+factor
    idx =0
    for i in range(int(n/2)+1+factor,n+1):
        if l[i+1]==l[i]:
            continue
        else:
            u2_merge.append(u2[temp])
            if temp==int(n/2)+1+factor:
                idx = len(u2_merge)
            temp = i+1
            
    u2_merge.append( -n-1-u2[0])
    weight2 = []
    for i in range(idx-1):
        weight2.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u2_merge[i]/2))
    weight2.append(1)
    for i in range(idx,len(l_merge)):
        weight2.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u2_merge[i]/2))

    totalWeight2 = sum(weight2)
    weight2 = weight2/totalWeight2
    i = np.random.choice(list(range(len(l_merge))), p=weight2)
    if i==idx:
        x2= l_merge[i]
    if i<idx:
        x2= np.random.randint(l_merge[i], l_merge[i+1])
    if i>idx:
        x2= np.random.randint(l_merge[i-1], l_merge[i])    
        

    indicator=0
    if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
        indicator=1
        
        
    return x1,x2 ,indicator,est
    

In [5]:
def SVTquantile(eps, beta, D, tau, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    T = tau

    for i in range(rad+1):
        Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i
def SVTmedian(eps, beta, D, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    n=len(D)
    T = int(n/2)

    for i in range(rad+1):
        Ti = T+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i
        
def SVTMedian(eps, beta, D):
    n = len(D)
    rad = 1000000
    Q = SVTmedian(eps, beta, D, rad)
    
    factor = int(16/eps*math.log(2*(2*rad+1)**2 *math.pi**2/(3*beta))+8/eps*math.log(4/beta))+1
    ##Then run SVT, do not double
    ##find the n/2 +- factor quantile
    T1 = int(n/2)-1
    T2 = int(n/2)+factor
    XLeft = SVTquantile(eps/2, beta/2, D, T1, rad)
    #print(XLeft)
    XRight = SVTquantile(eps/2, beta/2, D, T2, rad)
    #print(XRight)
    indicator=0
    if D[int(n/2)]<=Q+ XRight and D[int(n/2)]>=XLeft:
        indicator=1
    
    return Q, XLeft,XRight,indicator

In [6]:
def smoothMedian(eps, beta, D,delta=0):
    #given D, first compute S(D)
    n = len(D)
    m = int(n/2)
    rad = 1000000
    SS = 0
    if delta==0:
        alpha = eps/20
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)        
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))

        Q = D[int(n/2)]+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        factor = 20*noisySS/eps*inver_F(1-beta/4)
        if D[int(n/2)]<=Q+ factor and D[int(n/2)]>=Q-factor:
            indicator=1

        return Q, factor,indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS) 
        Q = D[int(n/2)]+4*SS/eps*LapNoise()
        
        
        indicator=0
        if D[int(n/2)]<=Q+ 4*noisySS/eps*math.log(2/beta) and D[int(n/2)]>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator
  

In [7]:
n=45211
D = np.random.uniform(0,40000, n)
D.sort()
D = discretize(D,1)
D[19000:21000]

array([16723., 16726., 16726., ..., 18526., 18526., 18526.])

In [8]:
print(D)
finiteDomainMedian(4,0.1,0,1000000, D)

[1.0000e+00 3.0000e+00 4.0000e+00 ... 3.9998e+04 3.9998e+04 3.9999e+04]


19937

In [9]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for EM based median, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        xl,xr,indi,est = EMMedian(eps[i], beta[3],  D)
        correct_count+=indi
        errors.append(abs(est-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)

Results for EM based median, Eps = 0.125
correct rate = 1.0
Average CI length = 1114.57
error quantile is 36.0
CI length/ error quantile = 30.960277777777776
Results for EM based median, Eps = 0.25
correct rate = 1.0
Average CI length = 558.76
error quantile is 18.0
CI length/ error quantile = 31.04222222222222
Results for EM based median, Eps = 0.5
correct rate = 1.0
Average CI length = 285.31
error quantile is 10.0
CI length/ error quantile = 28.531
Results for EM based median, Eps = 1
correct rate = 1.0
Average CI length = 146.615
error quantile is 5.0
CI length/ error quantile = 29.323
Results for EM based median, Eps = 2
correct rate = 1.0
Average CI length = 74.415
error quantile is 2.0
CI length/ error quantile = 37.2075
Results for EM based median, Eps = 4
correct rate = 1.0
Average CI length = 38.935
error quantile is 2.0
CI length/ error quantile = 19.4675
[1114.57, 558.76, 285.31, 146.615, 74.415, 38.935]
[36.0, 18.0, 10.0, 5.0, 2.0, 2.0]


In [10]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for EM based median, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        xl,xr,indi,est = EMMedian(eps[3], beta[i],  D)
        correct_count+=indi
        errors.append(abs(est-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for EM based median, beta = 0.01
correct rate = 1.0
Average CI length = 165.52
error quantile is 10.0
CI length/ error quantile = 16.552
Results for EM based median, beta = 0.025
correct rate = 1.0
Average CI length = 160.14
error quantile is 10.0
CI length/ error quantile = 16.014
Results for EM based median, beta = 0.05
correct rate = 1.0
Average CI length = 153.955
error quantile is 6.0
CI length/ error quantile = 25.659166666666668
Results for EM based median, beta = 0.1
correct rate = 1.0
Average CI length = 147.25
error quantile is 5.0
CI length/ error quantile = 29.45
Results for EM based median, beta = 0.2
correct rate = 1.0
Average CI length = 141.22
error quantile is 4.0
CI length/ error quantile = 35.305
[165.52, 160.14, 153.955, 147.25, 141.22]
[10.0, 10.0, 6.0, 5.0, 4.0]


In [10]:
#To save time, pre-compute and store all counts
rad = 1000000
counts = np.zeros(rad+1)
for i in range(rad-40000+1):
    counts[40000+i]=40000
    
    
temp_count=1
for i in range(39999):
    if D[i+1]==D[i]:
        temp_count+=1
    else:
        for j in range(int(D[i]), int(D[i+1])):
            counts[j]=temp_count
        temp_count+=1
        
        

SVTmedian(1, 0.1, D, rad)

19956

In [152]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for SVT based median, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,xl,xr,indi = SVTMedian(eps[i], beta[3], D)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)

Results for SVT based median, Eps = 0.125
correct rate = 1.0
Average CI length = 275.795
error quantile is 16.0
CI length/ error quantile = 17.2371875
Results for SVT based median, Eps = 0.25
correct rate = 1.0
Average CI length = 134.06
error quantile is 6.0
CI length/ error quantile = 22.343333333333334
Results for SVT based median, Eps = 0.5
correct rate = 1.0
Average CI length = 64.975
error quantile is 3.0
CI length/ error quantile = 21.65833333333333
Results for SVT based median, Eps = 1
correct rate = 1.0
Average CI length = 32.775
error quantile is 2.0
CI length/ error quantile = 16.3875
Results for SVT based median, Eps = 2
correct rate = 1.0
Average CI length = 16.615
error quantile is 1.0
CI length/ error quantile = 16.615
Results for SVT based median, Eps = 4
correct rate = 1.0
Average CI length = 9.495
error quantile is 1.0
CI length/ error quantile = 9.495
[275.795, 134.06, 64.975, 32.775, 16.615, 9.495]
[16.0, 6.0, 3.0, 2.0, 1.0, 1.0]


In [18]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for SVT based median, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,xl,xr,indi = SVTMedian(eps[3], beta[i], D)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for SVT based median, beta = 0.01
correct rate = 1.0
Average CI length = 254.87
error quantile is 18.0
CI length/ error quantile = 14.159444444444444
Results for SVT based median, beta = 0.025
correct rate = 1.0
Average CI length = 245.67
error quantile is 17.0
CI length/ error quantile = 14.451176470588235
Results for SVT based median, beta = 0.05
correct rate = 1.0
Average CI length = 239.15
error quantile is 12.0
CI length/ error quantile = 19.929166666666667
Results for SVT based median, beta = 0.1
correct rate = 1.0
Average CI length = 233.45
error quantile is 12.0
CI length/ error quantile = 19.454166666666666
Results for SVT based median, beta = 0.2
correct rate = 1.0
Average CI length = 226.415
error quantile is 7.0
CI length/ error quantile = 32.345
[254.87, 245.67, 239.15, 233.45, 226.415]
[18.0, 17.0, 12.0, 12.0, 7.0]


In [153]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for Smooth based median, pure DP"+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,length,indi = smoothMedian(eps[i], beta[3], D,delta=0)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)



Results for Smooth based median, pure DP0.125
correct rate = 0.97
Average CI length = 3209.4954949548696
error quantile is 2146.019647982299
CI length/ error quantile = 1.4955573673206848
Results for Smooth based median, pure DP0.25
correct rate = 0.96
Average CI length = 978.2441386929187
error quantile is 579.4031385573871
CI length/ error quantile = 1.6883652738377912
Results for Smooth based median, pure DP0.5
correct rate = 0.97
Average CI length = 287.65368667753404
error quantile is 155.4906160395003
CI length/ error quantile = 1.8499745772726213
Results for Smooth based median, pure DP1
correct rate = 0.98
Average CI length = 90.83257771106987
error quantile is 48.09590392887185
CI length/ error quantile = 1.8885720049133603
Results for Smooth based median, pure DP2
correct rate = 0.99
Average CI length = 29.85331961031193
error quantile is 15.568191377213225
CI length/ error quantile = 1.9175843157996821
Results for Smooth based median, pure DP4
correct rate = 0.99
Average CI 

In [179]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for Smooth based median, pure DP beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,length,indi = smoothMedian(eps[3], beta[i], D,delta=0)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for Smooth based median, pure DP beta = 0.01
correct rate = 1.0
Average CI length = 1163.4691618964418
error quantile is 517.8080749021319
CI length/ error quantile = 2.2469119704561247
Results for Smooth based median, pure DP beta = 0.025
correct rate = 1.0
Average CI length = 761.5676253910009
error quantile is 321.21578305097137
CI length/ error quantile = 2.3708910507369225
Results for Smooth based median, pure DP beta = 0.05
correct rate = 0.99
Average CI length = 573.372728778763
error quantile is 424.3672129168117
CI length/ error quantile = 1.3511240061120384
Results for Smooth based median, pure DP beta = 0.1
correct rate = 0.98
Average CI length = 417.93608563843947
error quantile is 204.59128050588697
CI length/ error quantile = 2.0427854237239287
Results for Smooth based median, pure DP beta = 0.2
correct rate = 0.96
Average CI length = 302.22116489699806
error quantile is 151.10884238158542
CI length/ error quantile = 2.00002303064316
[1163.4691618964418, 761.56762